# Migrating Pipelines between Platform Instances

Demonstrating how to get all the pipeline configuration details from a source organization and then create a new set of pipelines in a new organization.

Note: pipelines use a protocol (which uses task-scripts). the protocols of migrated pipelines need to be accessible in the destination org.

## Import Statements

In [ ]:
import os
import json
import requests

## Notebook Parameters
* SAVE_DIR = directory on your local machine

### Source Platform
* FILENAME_SOURCE = name for the authentication file, e.g. auth-dev.json
* API_URL_SOURCE = API URL for your TDP instance, e.g. https://api.tetrascience-dev.com/v1/
* AUTH_TOKEN_SOURCE = personal access token for TDP, or token of Service User
* TDP_ORG_SOURCE = organization slug for configuration report

### Destination Platform
* FILENAME_DEST = name for the authentication file, e.g. auth-prod.json
* API_URL_DEST = API URL for your TDP instance, e.g. https://api.tetrascience.com/v1/
* AUTH_TOKEN_DEST = personal access token for TDP, or token of Service User
* TDP_ORG_DEST = organization slug for configuration report

In [ ]:
SAVE_DIR = "./"

FILENAME_SOURCE = "auth-dev.json"
API_URL_SOURCE = "https://api.tetrascience-dev.com/v1/"
AUTH_TOKEN_SOURCE = ""
TDP_ORG_SOURCE = ""

FILENAME_DEST = "auth-prod.json"
API_URL_DEST = "https://api.tetrascience.com/v1/"
AUTH_TOKEN_DEST = ""
TDP_ORG_DEST = ""

## Create Authentication Files

This uses the notebook parameters to create authentication file, as outlined on documentation site for setting up your development environment: [Development Setup](https://developers.tetrascience.com/docs/set-up-your-environment-and-initialize-ts-sdk#set-up-the-environment)

This authentication file can also be used with our ts-sdk to deploy custom pipelines.

In [ ]:
def create_authentication_file(filename, save_directory, api_url, auth_token, tdp_org, ignore_ssl=False):
    auth_json_path = os.path.join(save_directory, filename)
    if ignore_ssl:
        ssl = "true"
    else:
        ssl = "false"
    with open(auth_json_path, "w") as f:
        auth_json = {"api_url": api_url,
                     "auth_token": auth_token,
                     "org": tdp_org,
                     "ignore_ssl": ssl}
        json.dump(auth_json, f, indent = 4)

In [ ]:
create_authentication_file(FILENAME_SOURCE, SAVE_DIR, API_URL_SOURCE, AUTH_TOKEN_SOURCE, TDP_ORG_SOURCE)

In [ ]:
create_authentication_file(FILENAME_DEST, SAVE_DIR, API_URL_DEST, AUTH_TOKEN_DEST, TDP_ORG_DEST)

## Use Authentication Files for API Headers

In [ ]:
with open(os.path.join(SAVE_DIR, FILENAME_SOURCE), "r") as f:
    auth_data_source = json.loads(f.read())

headers_source = {"ts-auth-token": auth_data_source["auth_token"],
               "x-org-slug": auth_data_source["org"]}

In [ ]:
with open(os.path.join(SAVE_DIR, FILENAME_DEST), "r") as f:
    auth_data_dest = json.loads(f.read())

headers_dest = {"ts-auth-token": auth_data_dest["auth_token"],
               "x-org-slug": auth_data_dest["org"],
               'Content-Type': 'application/json'}  # Sending pipeline details (json) to destination

In [ ]:
API_URL_SOURCE

## API Endpoints

In [ ]:
API_URL_SOURCE = auth_data_source["api_url"]
API_URL_DEST = auth_data_dest["api_url"]
PIPELINE_SEARCH = API_URL_SOURCE + "pipeline/search"
PIPELINE_CREATION = API_URL_DEST + "pipeline/create"

## Get all pipelines from source

Note: this only gets all enabled pipelines

In [ ]:
def get_pipeline_page(headers, **kwargs):
    """
        Returns a set of pipelines and whether there are more 
        pipelines remaining
        Optional args: page_size, page_index
    """
    pipeline_api = PIPELINE_SEARCH + "?"
    if "index" in kwargs.keys():
        page_index = kwargs["index"]
        pipeline_api += "from=" + str(page_index) + "&"
    if "size" in kwargs.keys():
        page_size = kwargs["size"]
        pipeline_api += "size=" + str(page_size) + "&"
    pipeline_api += "pipelineStatus=enabled"

    pipeline_response = requests.get(pipeline_api, headers=headers)
    pipeline_response = json.loads(pipeline_response.text)
    
    return pipeline_response["hits"], pipeline_response["hasNext"]

In [ ]:
def get_all_pipelines(headers, size=10):
    """
        Returns list of all pipelines by iterating over full list
        by the size parameter.
    """
    hasNext = True
    index = 0
    all_pipelines = []
    while hasNext == True:
        pipes, hasNext = get_pipeline_page(headers, size=size, index=index)
        all_pipelines += pipes
        index += 1
    return all_pipelines

In [ ]:
source_pipelines = get_all_pipelines(headers_source)

In [ ]:
source_pipelines

## Create all pipelines in destination

In [ ]:
type(source_pipelines[0])

In [ ]:
def create_all_pipelines(headers, pipelines):
    """
        Creates pipelines from a list of pipelines in provided org.
        For each pipeline, it will also print out the API call message.
    """
    for pipeline in pipelines:
        print("Copying %s" %pipeline["name"])
        payload = json.dumps(pipeline)
        api_call = requests.post(PIPELINE_CREATION, headers=headers, data=payload)
        print(api_call.text)

In [ ]:
create_all_pipelines(headers_dest, source_pipelines)